In [3]:
# Toy example given by gurobi

from gurobipy import *

try:
    # create a new model
    m = Model("mip1")

    # create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Establish objective

    m.setObjective(x+y+2*z, GRB.MAXIMIZE)

    # Add constraint 1
    m.addConstr(x+2*y+3*z<=4,"c0")
    # Add constraint 2
    m.addConstr(x+y>=1,"c1")
    m.optimize()
    for v in m.getVars():
        print(v.varName,v.x)
    print('Obj:', m.objVal)
except GurobiError:
    print('Error reported')

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1.0
y 0.0
z 1.0
Obj: 3.0
